# 1. Setup Mario

In [60]:
!pip install gym_super_mario_bros
!pip install nes_py

In [61]:
# Import the game
import gym_super_mario_bros
# Import the oJoypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [62]:
# Important simplify game environnement

# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [63]:
env.observation_space.shape

(240, 256, 3)

In [64]:
env.action_space

Discrete(7)

In [65]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [66]:
done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()

ValueError: not enough values to unpack (expected 5, got 4)